In [1]:
pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.


# MODEL 1

In [2]:
import pdb
import re
import sys
import torch.optim as optim
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertPreTrainedModel, BertModel, BertForSequenceClassification, BertTokenizer
import torch
from torch import nn
import torch.nn.functional as F
from torch.distributions import Categorical
import random
import numpy as np
import os
import copy
import time, json
import datetime
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, random_split, Subset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.autograd import Variable
from sklearn.metrics import f1_score, precision_recall_fscore_support
from collections import Counter
import csv
import argparse
import pandas as pd
from transformers import AutoModel
print("COMPLETE")

COMPLETE


In [3]:
from transformers import BigBirdTokenizer
from transformers import BigBirdModel
import sentencepiece
print("COMPLETE")

COMPLETE


In [4]:
import re
import nltk

nltk.download('punkt')

def format_evidence(evidence):
    full_text_list = [item['text'] for item in evidence.values()]
    full_text = ' '.join(full_text_list)  # 띄어쓰기 유지하며 문자열 합치기
    
    # 문장 단위로 분할 (NLTK 활용)
    sentences = nltk.sent_tokenize(full_text)
    
    # 너무 짧은 문장 제거 (5자 이하)
    sentences = [s.strip() for s in sentences if len(s.strip()) > 5][:25]
    
    # 빈 리스트 예외 처리
    if not sentences:
        return ''
    
    return ' [SEP] '.join(sentences) + ' [SEP] '

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
data_wiki = pd.read_csv('/kaggle/input/gold0303/labeled_gold_0302.csv')
data_wiki.head()

,Unnamed: 0.1,Unnamed: 0,news,extracted_entities,wiki_summary,wiki_full,label,title,news_merged,key sentences,key sentences explanation,summarized evidence sentences,gold
0,0,0,u.s. senate republican leader mitch mcconnell ...,"Mitch McConnell, Myanmar, Aung San Suu Kyi, Jo...",Mitch McConnell: Addison Mitchell McConnell II...,Mitch McConnell: Addison Mitchell McConnell II...,1,senate leader opposes lecturing myanmar leader...,title : senate leader opposes lecturing myanma...,"[""Mitch McConnell said on Tuesday he would not...","[""McConnell's opposition to the resolution dir...","[""Mitch McConnell is an American politician an...","[2, 3, 8, 6, 9, 14]"
1,1,2,note little johnny might want consider use dia...,"little johnny, isis, fort riley military base,...",little johnny: Little Johnny jokes are about a...,little johnny: Little Johnny jokes are about a...,1,NaN,content : note little johnny might want consid...,"[""A 20-year-old Kansas man plotted to kill an ...","[""This sentence supports the news because it p...","[""Little Johnny jokes are about a small boy wh...","[13, 12, 29, 30]"
2,2,3,president obama is clearly in good spirits tha...,"president obama, republican party, trump presi...",president obama: Barack Hussein Obama II (born...,president obama: Barack Hussein Obama II (born...,0,president obama gets his troll game on by than...,title : president obama gets his troll game on...,"[""Obama said it was crucial for voters to oppo...","[""This sentence supports the news because it e...","[""President Obama is the 44th president of the...","[13, 14, 4]"
3,3,5,a racist white woman in florida is surely furi...,"Florida, Tallahassee, Trump, Martin Luther Kin...",Florida: Florida ( FLORR-ih-də; Spanish: [flo...,Florida: Florida ( FLORR-ih-də; Spanish: [flo...,0,racist florida woman tells black people you sh...,title : racist florida woman tells black peopl...,[],[],"[""Florida is a state in the Southeastern regio...",[]
4,4,6,anyone question whether bundy-led right-wing t...,"Bundy, Malheur National Wildlife Refuge, Amand...",Bundy: Theodore Robert Bundy (né Cowell; Novem...,Bundy: Theodore Robert Bundy (né Cowell; Novem...,1,NaN,content : anyone question whether bundy-led ri...,"[""On January 2, 2016, an armed group of right-...","[""This sentence supports the news because it f...","[""On January 2, 2016, an armed group of right-...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]"


In [6]:
data_wiki['summarized evidence sentences'][0]

'["Mitch McConnell is an American politician and the senior United States senator from Kentucky.", "He has served from 2007 to 2025 as the leader of the Senate Republican Conference.", "On Tuesday, McConnell stated he would not support a resolution targeting Myanmar leader Aung San Suu Kyi.", "He expressed that he does not favor a resolution going after her.", "McConnell has been engaged with Myanmar issues for years.", "He regards Aung San Suu Kyi as the greatest hope for Myanmar\'s transition from military dictatorship.", "Senators John McCain and Richard Durbin introduced a resolution condemning violence against the Rohingya Muslims.", "McConnell expressed his disagreement with McCain and Durbin\'s resolution.", "He commented that America lecturing Aung San Suu Kyi in her challenging position is unhelpful.", "International pressure on Myanmar has increased regarding the Rohingya Muslims\' treatment.", "Approximately 370,000 Rohingya Muslims have fled to Bangladesh due to violence.",

In [7]:
import ast

s = data_wiki['summarized evidence sentences'][0]
k = data_wiki['key sentences'][0]
# 문자열을 리스트로 변환
lsts = ast.literal_eval(s)
lstk = ast.literal_eval(k)
print(len(lsts))
print(lsts)

31
['Mitch McConnell is an American politician and the senior United States senator from Kentucky.', 'He has served from 2007 to 2025 as the leader of the Senate Republican Conference.', 'On Tuesday, McConnell stated he would not support a resolution targeting Myanmar leader Aung San Suu Kyi.', 'He expressed that he does not favor a resolution going after her.', 'McConnell has been engaged with Myanmar issues for years.', "He regards Aung San Suu Kyi as the greatest hope for Myanmar's transition from military dictatorship.", 'Senators John McCain and Richard Durbin introduced a resolution condemning violence against the Rohingya Muslims.', "McConnell expressed his disagreement with McCain and Durbin's resolution.", 'He commented that America lecturing Aung San Suu Kyi in her challenging position is unhelpful.', "International pressure on Myanmar has increased regarding the Rohingya Muslims' treatment.", 'Approximately 370,000 Rohingya Muslims have fled to Bangladesh due to violence.', 

In [8]:
def strtolist(text):
    lsti = ast.literal_eval(text)
    return lsti

data_wiki['gold'] = data_wiki['gold'].apply(strtolist)
data_wiki['summarized evidence sentences'] = data_wiki['summarized evidence sentences'].apply(strtolist)

bigbird -> MLP -> golden evidence 예측

In [9]:
max = 0
for i in range(1385):
    for j in data_wiki['gold'][i]:
        if j > max:
            max = j

print(max)

35


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [11]:
import re
import nltk

nltk.download('punkt')

def format_evidence(evidence):
    full_text_list = nltk.sent_tokenize(evidence)
    
    # 하나의 문자열로 합치기
    full_text = ' '.join(full_text_list)

    # 문장 분할 (NLTK 사용)
    sentences = nltk.sent_tokenize(full_text)

    # 너무 짧은 문장 제거 (5자 이하)
    sentences = [s.strip() for s in sentences if len(s.strip()) > 5][:25]

    # 빈 리스트 예외 처리
    if not sentences:
        return ''

    # [SEP] 추가하여 하나의 문자열로 변환
    return ' [SEP] '.join(sentences) + ' [SEP] '

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
#claims
claims = []
for a in data_wiki['news']:
    claims.append(a)

In [13]:
from sklearn.model_selection import train_test_split 

import random

gold_evidence_mask = []

def to_one(size, indices):
    return [1 if i in indices else 0 for i in range(size)]

for i in range(len(data_wiki['gold'])):
    gold_evidence_mask.append(to_one(40,data_wiki['gold'][i]))



sentences = []                                                     #판별할 문장
for claim, evidence in zip(claims, data_wiki['summarized evidence sentences']):
    sentence = f" [CLS] {claim} [SEP] {evidence} [SEP]"
    sentences.append(sentence)


data = pd.DataFrame(sentences, columns=["label"]) # [CLS] claim [SEP] evidence [SEP]
data['label'] = gold_evidence_mask # 정답 Gold evidence들의 top 5 indices
data['evidences'] = data_wiki['summarized evidence sentences']
data['sentences'] = sentences
data

,label,evidences,sentences
0,"[0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, ...",[Mitch McConnell is an American politician and...,[CLS] u.s. senate republican leader mitch mcc...
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, ...",[Little Johnny jokes are about a small boy who...,[CLS] note little johnny might want consider ...
2,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...",[President Obama is the 44th president of the ...,[CLS] president obama is clearly in good spir...
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[Florida is a state in the Southeastern region...,[CLS] a racist white woman in florida is sure...
4,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ...","[On January 2, 2016, an armed group of right-w...",[CLS] anyone question whether bundy-led right...
...,...,...,...
1381,"[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, ...",[Donald John Trump is an American politician a...,[CLS] washington u.s. senate expect vote 11 a...
1382,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, ...",[The Federal Bureau of Investigation (FBI) is ...,[CLS] this is huge! the doj has authorized th...
1383,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[Paul Ryan is an American politician who serve...,[CLS] washington u.s. house speaker paul ryan...
1384,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, ...",[Dinesh D'Souza warned about Obama's reduction...,[CLS] dinesh d sousa warned us about obama s ...


In [14]:
#train for sentence
from sklearn.model_selection import train_test_split

train_text, temp_text, train_label, temp_label = train_test_split(data['sentences'], data['label'],
                                                                  test_size=0.3,
                                                                  random_state=2018
                                                                  )

val_text, test_text, val_label, test_label = train_test_split(temp_text, temp_label,
                                                                  test_size=0.5,
                                                                  random_state=2018
                                                                  )

print_text, _, print_label, _ = train_test_split(temp_text, temp_label,
                                                test_size=0.99,
                                                random_state=2018
                                                )

In [15]:
print(train_text.index) #Loss function에 사용될 실제 데이터 index들.

Index([ 953,   36, 1150,  401,  167, 1365,  324,  305,  190,  910,
       ...
         87,  537,  902,  148, 1180,  917,  777,  226,  614, 1274],
      dtype='int64', length=970)


In [ ]:
#model2
class NewsVerificationModel(nn.Module):
    def __init__(self, model_name="google/bigbird-roberta-base", freeze_bert=False):
        """
        뉴스 진위여부 판별 모델 (Cross Attention 활용)
        Args:
            model_name: 사용할 사전학습 언어모델 이름
            freeze_bert: 언어모델 파라미터 동결 여부
        """
        super(NewsVerificationModel, self).__init__()

        # (1) 언어모델
        self.bert = AutoModel.from_pretrained(model_name)
        self.hidden_size = self.bert.config.hidden_size

        # (2) 동결 설정
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False

        # (3) 문장 레벨 어텐션
        self.sentence_attention = SentenceWiseAttention(self.hidden_size)

        # (4) 뉴스-문서 Cross Attention
        self.cross_attention = nn.MultiheadAttention(
            embed_dim=self.hidden_size,
            num_heads=8,
            batch_first=True
        )

        # (5) 최종 분류기
        self.classifier = nn.Sequential(
            nn.Linear(self.hidden_size * 2, 256),  # (뉴스CLS + 문서임베딩)
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(64, 2)  # (진실 / 거짓)
        )

    def forward(self,
                news_input_ids, news_attention_mask,
                doc_sentences_input_ids, doc_sentences_attention_mask):
        """
        Args:
            news_input_ids: (batch_size, news_len)
            news_attention_mask: (batch_size, news_len)
            doc_sentences_input_ids: (batch_size, num_sentences, sentence_len)
            doc_sentences_attention_mask: (batch_size, num_sentences, sentence_len)
        Returns:
            logits: (batch_size, 2)
            sentences_attention_weights: (batch_size, num_sentences)
        """
        device = news_input_ids.device
        batch_size, num_sentences, sent_len = doc_sentences_input_ids.shape

        # ---------------------
        # 1) 뉴스 인코딩
        # ---------------------
        news_outputs = self.bert(
            news_input_ids,
            attention_mask=news_attention_mask,
            return_dict=True
        )
        # 전체 임베딩 (batch_size, news_len, hidden_size)
        news_emb = news_outputs.last_hidden_state
        # 뉴스 CLS (batch_size, hidden_size)
        news_cls = news_emb[:, 0, :]

        # ---------------------
        # 2) 문서 문장 인코딩
        # ---------------------
        # Flatten
        flattened_ids = doc_sentences_input_ids.view(-1, sent_len)           # (batch_size * num_sentences, sentence_len)
        flattened_mask = doc_sentences_attention_mask.view(-1, sent_len)     # 동일 크기

        valid_sent_mask = flattened_mask.sum(dim=1) > 0
        all_embeddings = torch.zeros(
            (batch_size * num_sentences, self.hidden_size),
            device=device
        )
        if valid_sent_mask.sum() > 0:
            valid_ids = flattened_ids[valid_sent_mask]
            valid_mask = flattened_mask[valid_sent_mask]

            valid_out = self.bert(
                valid_ids,
                attention_mask=valid_mask,
                return_dict=True
            )
            # (유효 문장의 CLS)
            valid_cls = valid_out.last_hidden_state[:, 0, :]
            all_embeddings[valid_sent_mask] = valid_cls

        # 다시 (batch_size, num_sentences, hidden_size) 형태로 복원
        sentence_embeddings = all_embeddings.view(batch_size, num_sentences, self.hidden_size)

        # ---------------------
        # 3) Cross Attention
        # ---------------------
        # Query: sentence_embeddings
        # Key, Value: news_emb
        # key_padding_mask는 True가 '무시할 위치' 이므로
        # news_attention_mask: 1=사용,0=패딩 => ~news_attention_mask.bool()

        key_padding_mask = ~news_attention_mask.bool()  # shape: (batch_size, news_len)

        cross_attn_out, _ = self.cross_attention(
            query=sentence_embeddings,         # (batch_size, num_sentences, hidden_size)
            key=news_emb,                      # (batch_size, news_len, hidden_size)
            value=news_emb,                    # (batch_size, news_len, hidden_size)
            key_padding_mask=key_padding_mask, # (batch_size, news_len)
            need_weights=False
        )
        # cross_attn_out: (batch_size, num_sentences, hidden_size)

        # ---------------------
        # 4) 문장별 어텐션
        # ---------------------
        # (batch_size, num_sentences)
        sentence_exists_mask = (doc_sentences_attention_mask.sum(dim=2) > 0)

        doc_embedding, sentences_attention_weights = self.sentence_attention(
            cross_attn_out,
            attention_mask=sentence_exists_mask
        ) # 가중합 반환

        # ---------------------
        # 5) 최종 분류
        # ---------------------
        combined = torch.cat([news_cls, doc_embedding], dim=1)  # (batch_size, hidden_size*2)
        logits = self.classifier(combined)                      # (batch_size, 2)

        return logits, sentences_attention_weights

model2 = NewsVerificationModel()

In [20]:
#claim & gold evidence labels (claim은 안쓰임 이미 sentence에 들어갔기 때문에)
train_claim, temp_claim, train_label, temp_label = train_test_split(data_wiki['news'], data['label'],
                                                                  test_size=0.3,
                                                                  random_state=2018
                                                                  )
val_claim, test_claim, val_label, test_label = train_test_split(temp_claim, temp_label,
                                                                  test_size=0.5,
                                                                  random_state=2018
                                                                  )


In [21]:
train_evidence, temp_evidence, train_nothing, temp_nothing = train_test_split(data_wiki['wiki_summary'], data['label'],
                                                                              test_size=0.3,
                                                                              random_state=2018)
val_evidence, test_evidence, nothing1, nothing2 = train_test_split(temp_evidence, temp_nothing,
                                                                              test_size=0.5,
                                                                              random_state=2018)

In [22]:
#학습 데이터 로딩
import torch
from torch.utils.data import Dataset, DataLoader

class FakeNewsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=4096):
        self.sending = texts.index
        self.texts = texts.tolist()  # Claim + Evidence
        self.labels = labels.tolist()  # 50개의 0과 1 벡터
        self.tokenizer = tokenizer
        self.max_length = max_length
         

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx], 
            padding="max_length",
            truncation=True, 
            max_length=self.max_length, 
            return_tensors="pt",
            add_special_tokens=True,
        )
        input_ids = encoding["input_ids"].squeeze(0)
        attention_mask = encoding["attention_mask"].squeeze(0)
        label = torch.tensor(self.labels[idx], dtype=torch.float)
        k = self.sending[idx] # Loss function에 갖다줄 index

        return input_ids, attention_mask, label, k


In [32]:
tokenizer = BigBirdTokenizer.from_pretrained("google/bigbird-roberta-base")

train_dataset = FakeNewsDataset(train_text, train_label, tokenizer)
val_dataset = FakeNewsDataset(val_text, val_label, tokenizer)
test_dataset = FakeNewsDataset(test_text, test_label, tokenizer)
print_dataset = FakeNewsDataset(print_text, print_label, tokenizer)

In [24]:
import gc

import gc
import torch.nn as nn


gc.collect()  # Python의 가비지 컬렉션 실행
torch.cuda.empty_cache()

In [25]:
#Model 1
import torch
import torch.nn as nn
from transformers import BigBirdModel

class BigBirdWithMLP(nn.Module):
    def __init__(self, model_name="google/bigbird-roberta-base", output_size=40, freeze_bigbird=False):
        super(BigBirdWithMLP, self).__init__()
        self.bigbird = BigBirdModel.from_pretrained(model_name, attention_type="block_sparse")
        self.hidden_size = self.bigbird.config.hidden_size  # 768
        
        self.mlp = nn.Sequential(
            nn.Linear(768, 128),
            nn.ReLU(),
            nn.LayerNorm(128),  # ✅ 속도 최적화
            nn.Dropout(0.2),  # ✅ 과적합 방지
            nn.Linear(128, 40),
            nn.Sigmoid()
        )
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bigbird(input_ids=input_ids, attention_mask=attention_mask)
        cls_token = outputs.last_hidden_state[:, 0, :]  # [CLS] 토큰 추출
        logits = self.mlp(cls_token)  # MLP 통과
        return logits  # (batch_size, 40) → 확률값이 아니라 logits 반환

In [26]:
import torch
import torch.nn as nn

# Loss_fullness, Loss_sufficiency를 반영한 Loss function (ReRead 논문)
class CustomSentenceRemovalLoss(nn.Module):
    def __init__(self):
        super(CustomSentenceRemovalLoss, self).__init__()

    def forward(self, predictions, targets, input_ids, attention_masks, indices):
        """
        predictions: 모델의 출력 (0~1 확률값, batch_size x 50)
        targets: 정답 라벨 (0 또는 1, batch_size x 50)
        """
        loss = 0
        final_loss = 0

        size = len(predictions) #batch의 크기

        for i in range(size):
            with torch.no_grad():
                outputs = model2(input_ids=input_ids[i].unsqueeze(0).to(device), 
                                attention_mask=attention_masks[i].unsqueeze(0).to(device))
                probs = torch.sigmoid(outputs.logits)
                og_value = data_wiki['label'].iloc[indices[i].item()] # 아무 조작도 가하지 않은 문장의 pure 예측
    
                del outputs
                torch.cuda.empty_cache()
                
                top5_indices = torch.topk(predictions[i], 5).indices #Model 1 이 예측한 evidence 문서안의 top 5 index
                top5_indices = set(top5_indices.tolist())
                evidences = data['evidences'].iloc[indices[i].item()] # evidence 문서를 아까 각 문장으로 쪼개놓았는데, 2D list에서 가져온 evidence 문서
                                                                      # 여기서 아까 FakeNewsDataset의 k값이 사용되는 것임
        
                fullness_sentence = " ".join([evidences[i] for i in range(len(evidences)) if i not in top5_indices]) # 예측한 top 5를 뺀문장
                suff_sentence = " ".join([evidences[i] for i in range(len(evidences)) if i in top5_indices]) # 예측한 top 5만 있는문장
        
                inputs = tokenizer(fullness_sentence, padding="max_length", truncation=True, max_length=4096, return_tensors="pt").to(device)
                outputs = model2(**inputs)
                probs = torch.sigmoid(outputs.logits)
                fullness_value = probs[0, 1].item()
                
                del outputs
                torch.cuda.empty_cache()
                
                inputs = tokenizer(suff_sentence, padding="max_length", truncation=True, max_length=4096, return_tensors="pt").to(device)
                outputs = model2(**inputs)
                probs = torch.sigmoid(outputs.logits)
                suff_value = probs[0, 1].item()
    
                del outputs 
                torch.cuda.empty_cache()

                # 그 두개의 문장으로 얻은 두가지 Loss(fullness, sufficiency)
                Loss_fullness = abs(og_value - fullness_value)
                Loss_suff = abs(og_value - suff_value)

                # fullness는 클수록 좋기 때문에 -를 붙이고, suff는 작을수록 좋기 때문에 + -> 최종 loss값은 작아야 좋다는 일관성 확보
                loss = - Loss_fullness + Loss_suff
                final_loss += loss
                

        final_loss = torch.tensor(final_loss, requires_grad=True, device="cuda")
        return final_loss/size #loss의 평균값 전달

In [ ]:
# # ✅ 학습 함수 without full suff loss
# def train(model, train_loader, val_loader, optimizer, criterion, epochs=30, device="cuda"):
#     model.to(device)

#     best_loss = float("inf")
#     best_model_path = "/kaggle/working/best_model.pt"
    
#     for epoch in range(epochs):
#         model.train()
#         total_loss = 0
        
#         for batch in tqdm(train_loader):
#             input_ids, attention_mask, labels, _ = [x.to(device) for x in batch]

#             optimizer.zero_grad()
#             outputs = model(input_ids, attention_mask)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()
#             total_loss += loss.item()
        
#         avg_train_loss = total_loss / len(train_loader)
#         print(f"Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f}")
        
#         # Validation
#         model.eval()
#         total_val_loss = 0
#         with torch.no_grad():
#             for batch in val_loader:
#                 input_ids, attention_mask, labels, _ = [x.to(device) for x in batch]
#                 outputs = model(input_ids, attention_mask)
#                 loss = criterion(outputs, labels)
#                 total_val_loss += loss.item()

        
#         avg_val_loss = total_val_loss / len(val_loader)
#         print(f"Epoch {epoch+1}: Validation Loss = {avg_val_loss:.4f}")

#         if avg_val_loss < best_loss:
#                             best_loss = avg_val_loss
#                             torch.save(model.state_dict(), best_model_path)
#                             print(f"✅ Best Model Saved at {best_model_path} (Loss: {best_loss:.4f})")

# # ✅ 데이터 로드 및 학습 실행
# tokenizer = BigBirdTokenizer.from_pretrained("google/bigbird-roberta-base")

# train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

# # 모델 및 옵티마이저 설정
# model = BigBirdWithMLP()
# optimizer = torch.optim.AdamW([
#     {"params": model.bigbird.parameters(), "lr": 2e-5},  # BigBird는 작은 lr
#     {"params": model.mlp.parameters(), "lr": 1e-3}  # MLP는 큰 lr
# ])
# criterion = CustomSentenceRemovalLoss()

# # 학습 실행
# train(model, train_loader, val_loader, optimizer, criterion, epochs=30)

In [38]:
from sklearn.metrics import accuracy_score

model.eval()
total_test_loss = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader):
        input_ids, attention_mask, labels, _ = [x.to(device) for x in batch]
        
        # 모델 예측값 (logits)
        outputs = model(input_ids, attention_mask)
        
        # 손실 계산
        loss = criterion(outputs, labels)
        total_test_loss += loss.item()

        # ✅ Sigmoid 적용 후 0.5 기준으로 예측 변환
        preds = (torch.sigmoid(outputs) > 0.5).int()
        
        # 리스트에 저장 (CPU로 변환)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 평균 Loss 계산
avg_test_loss = total_test_loss / len(test_loader)

# 정확도 계산
accuracy = accuracy_score(all_labels, all_preds)

print(f"Accuracy: {accuracy:.4f} %")

  0%|          | 0/21 [00:00<?, ?it/s]

Accuracy: 71.0247 %


In [69]:
def highlight_indices(lst, highlight_idxs, color="\033[91m"):  # 91 = 빨간색
    reset = "\033[0m"  # 색상 초기화
    result = []
    
    for i, val in enumerate(lst):
        if i in highlight_idxs:
            result.append(f"{color}{val}{reset}")  # 특정 인덱스 색상 적용
        else:
            result.append(str(val))
    
    print("[" + ", ".join(result) + "]")

In [ ]:
# Loss_fullness & Loss_efficiency
def train(model, train_loader, val_loader, optimizer, criterion, epochs=3, device="cuda"):
    model.to(device)
    for param in model.bigbird.parameters():
        param.requires_grad = False
    
    for epoch in range(epochs):
        model.train()        
        total_loss = 0
        
        for batch in tqdm(train_loader):
            input_ids, attention_mask, labels, indices = [x.to(device) for x in batch]

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels, input_ids,attention_mask, indices)# 여기서 Input_ids랑 indices를 lossfunction에게 전달해서 사용할 수 있게끔
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f}")
        
        # Validation
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                input_ids, attention_mask, labels, indices = [x.to(device) for x in batch]
                outputs = model(input_ids, attention_mask)
                loss = criterion(outputs, labels, input_ids,attention_mask, indices)
                total_val_loss += loss.item()
        
        avg_val_loss = total_val_loss / len(val_loader)
        print(f"Epoch {epoch+1}: Validation Loss = {avg_val_loss:.4f}")

        if avg_val_loss < best_loss:
                    best_loss = avg_val_loss
                    torch.save(model.state_dict(), best_model_path)
                    print(f"✅ Best Model Saved at {best_model_path} (Loss: {best_loss:.4f})")

# ✅ 데이터 로드 및 학습 실행
tokenizer = BigBirdTokenizer.from_pretrained("google/bigbird-roberta-base")

train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=10, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

# 모델 및 옵티마이저 설정
model = BigBirdWithMLP()
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
criterion = 0.7 * CustomSentenceRemovalLoss() + 0.3 * nn.BCEWithLogitsLoss()

# 학습 실행
train(model, train_loader, val_loader, optimizer, criterion, epochs=3)